In [3]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from keras.models import model_from_json 

Using TensorFlow backend.


In [19]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)

    #Extract relevant DICOM tags
    image_position = ds.get('PatientPosition', None)
    image_type = ds.get('Modality', None)
    body_part = ds.get('BodyPartExamined', None)
    
    # Validate PatientPosition: must be 'PA' or 'AP'
    # Validate Modality: must be 'DX'
    # Validate Body Part: must be 'CHEST'

    if image_position in ['PA', 'AP'] and image_type == 'DX' and body_part == 'CHEST':
        return ds.pixel_array
    else:
        print(f"Unsupported Image Type (Modality): {image_type}, (BodyPartExamined): {body_part}, (Image Position): {image_position}")

    return 
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    
    img_norm = img/255.0
    new_img = (img_norm - img_mean)/img_std
    proc_img = np.resize(new_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    
    with open(model_path, 'r') as json_file:
        json_savedModel= json_file.read() 
        
    model = model_from_json(json_savedModel)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_image = model.predict(img)
    prediction = 1 if pred_image >= thresh else 0
    
    return prediction 

In [21]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'pneumonia_detection_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.49731431 # loads the mean image value they used during training preprocessing
img_std = 0.25106729 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh = 0.47 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue

    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    if pred:
        print(f'{i} has Pneumonia')
    else:
        print(f'{i} does not have Pneumonia')

Load file test1.dcm ...
test1.dcm does not have Pneumonia
Load file test2.dcm ...
test2.dcm has Pneumonia
Load file test3.dcm ...
test3.dcm has Pneumonia
Load file test4.dcm ...
Unsupported Image Type (Modality): DX, (BodyPartExamined): RIBCAGE, (Image Position): PA
Load file test5.dcm ...
Unsupported Image Type (Modality): CT, (BodyPartExamined): CHEST, (Image Position): PA
Load file test6.dcm ...
Unsupported Image Type (Modality): DX, (BodyPartExamined): CHEST, (Image Position): XX
